This notebook is for running a first try of BOWave on the Frolich et. al data to see if we match their paper's results.
This requires 16 < x < 32 gb of RAM. Recommend running on Caviness with --mem=32gb flag set.

#Load ICs

In [ ]:
import BOWaves.utilities.dataloaders as dataloaders
import os
import numpy as np

In [ ]:
#frolich_ics = {'ICs': np.array([]), 'labels': np.array([])}
frolich_ics = {'ICs': [], 'labels': []}

#for file in directory frolich data
frolich_data = os.listdir('../data/frolich')

#filter out subdirectories such as /img
frolich_data = [file for file in frolich_data if not os.path.isdir(file)]

for file in frolich_data:
    ICs, labels = dataloaders.load_and_visualize_mat_file_frolich('../data/frolich/' + file, visualize=False)
    frolich_ics['ICs'].extend(ICs)
    frolich_ics['labels'].extend(labels)


Now create codebooks since we have the ICs and their labels.

First split off 20% for testing.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(frolich_ics['ICs'], frolich_ics['labels'], test_size=0.2, random_state=42)

Now out of the training set, split into the different classes. Frolich's data has 4 classes.

In [ ]:
if len(X_train) != len(y_train):
    raise ValueError('X_train and y_train are not the same length.')

# Forgot what the classes were. check on Caviness

for i in range(len(X_train)):
    if y_train[i] == 1:
        class1.append(X_train[i])
    elif y_train[i] == 2:
        class2.append(X_train[i])
    elif y_train[i] == 3:
        class3.append(X_train[i])
    elif y_train[i] == 4:
        class4.append(X_train[i])
    else:
        raise ValueError('y_train has a value that is not 1, 2, 3, or 4.')

In [ ]:
metric, init = 'cosine', 'random'

#need to do this per class.
centroids, labels, shifts, distances, inertia, _ = shift_invariant_k_means(
    X_train, args.num_clusters, args.centroid_len, metric=metric, init=init, n_init=args.n_runs, rng=rng,  verbose=True, n_jobs=args.n_jobs)